### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')


### CME Data

In [3]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)

In [5]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [6]:
# Preview the first result in JSON format
print(cme_json[0])
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{'activityID': '2013-05-01T03:12:00-CME-001', 'catalog': 'M2M_CATALOG', 'startTime': '2013-05-01T03:12Z', 'instruments': [{'displayName': 'SOHO: LASCO/C2'}, {'displayName': 'SOHO: LASCO/C3'}, {'displayName': 'STEREO A: SECCHI/COR2'}, {'displayName': 'STEREO B: SECCHI/COR2'}], 'sourceLocation': '', 'activeRegionNum': None, 'note': '', 'submissionTime': '2013-08-07T16:54Z', 'versionId': 1, 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1', 'cmeAnalyses': [{'isMostAccurate': True, 'time21_5': '2013-05-01T07:07Z', 'latitude': 12.0, 'longitude': -120.0, 'halfAngle': 36.0, 'speed': 860.0, 'type': 'C', 'featureCode': 'null', 'imageType': None, 'measurementTechnique': 'null', 'note': '', 'levelOfData': 0, 'tilt': None, 'minorHalfWidth': None, 'speedMeasuredAtHeight': None, 'submissionTime': '2013-08-07T16:54Z', 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CMEAnalysis/2350/-1', 'enlilList': [{'modelCompletionTime': '2013-05-01T08:32Z', 'au': 2.0, 'estimatedShockArr

In [7]:
# Convert cme_json to a Pandas DataFrame 

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = pd.DataFrame(cme_json, columns=["activityID", "startTime", "linkedEvents"])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [8]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=["linkedEvents"])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [9]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 
# Initialize an empty list to store the expanded rows
# Iterate over each index in the DataFrame
# Iterate over each dictionary in the list
# Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
# Create a new DataFrame from the expanded rows

expanded_rows = [{'activityID': row['activityID'], 'startTime': row['startTime'], 'linkedEvents': linked_event} for index, row in cme_df.iterrows() for linked_event in row["linkedEvents"]]
cme_df_expanded = pd.DataFrame(expanded_rows)
cme_df_expanded.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [10]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
# Log the error or print it for debugging
def extract_activityID_from_dict(linked_event):
        try:
                return linked_event["activityID"]
        except KeyError as e:
                print(f"Error: {e} - skipping")
                return None


In [11]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df_expanded["GST_ActivityID"] = cme_df_expanded["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
cme_df_expanded.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df_expanded = cme_df_expanded.dropna(subset=["GST_ActivityID"])

In [13]:
# print out the datatype of each column in this DataFrame:
cme_df_expanded.dtypes

activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object

In [34]:
# Make a copy of the cme_df_expanded DataFrame
cme_df_copy = cme_df_expanded.copy()
# Convert the 'GST_ActivityID' column to string format 
cme_df_copy["GST_ActivityID"] = cme_df_expanded["GST_ActivityID"].astype(str)
# Convert the 'activityID' column to string format 
cme_df_copy['activityID'] = cme_df_expanded['activityID'].astype(str)
# Convert startTime to datetime format  
cme_df_copy["startTime"] = pd.to_datetime(cme_df_expanded["startTime"])
# Rename startTime to startTime_CME and activityID to cmeID
cme_df_copy = cme_df_copy.rename(columns={"startTime": "startTime_CME", "activityID": "cmeID"})

# Drop linkedEvents
cme_df_copy = cme_df_copy.drop(columns=["linkedEvents"])
# Verify that all steps were executed correctly
cme_df_copy.dtypes

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object

In [15]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df_copy = cme_df_copy[cme_df_copy["GST_ActivityID"].str.contains("GST")]
cme_df_copy.head()

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [16]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}GST?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [17]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [18]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
print(gst_json[0])
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

{'gstID': '2013-06-01T01:00:00-GST-001', 'startTime': '2013-06-01T01:00Z', 'allKpIndex': [{'observedTime': '2013-06-01T01:00Z', 'kpIndex': 6.0, 'source': 'NOAA'}], 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1', 'linkedEvents': [{'activityID': '2013-05-31T15:45:00-HSS-001'}], 'submissionTime': '2013-07-15T19:26Z', 'versionId': 1}
{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [19]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json, columns=["gstID", "startTime", "linkedEvents"])
# Keep only the columns: activityID, startTime, linkedEvents
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [20]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=["linkedEvents"])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [21]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode("linkedEvents").reset_index(drop=True).dropna(subset=["linkedEvents"])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [22]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df["CME_ActivityID"] = gst_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset=["CME_ActivityID"])
gst_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [35]:
# Make a copy of the cme_df_expanded DataFrame
gst_df_copy = gst_df.copy()
# Convert the 'CME_ActivityID' column to string format 
gst_df_copy['CME_ActivityID'] = gst_df_copy['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_df_copy['gstID'] = gst_df_copy['gstID'].astype(str)
# Convert startTime to datetime format  
gst_df_copy['startTime'] = pd.to_datetime(gst_df['startTime'])
# Rename startTime to startTime_GST 
gst_df_copy = gst_df_copy.rename(columns={'startTime': "startTime_GST"})
# Drop linkedEvents
gst_df_copy = gst_df_copy.drop(columns=['linkedEvents'])
# Verify that all steps were executed correctly
gst_df_copy.dtypes

gstID                          object
startTime_GST     datetime64[ns, UTC]
CME_ActivityID                 object
dtype: object

In [31]:
gst_df_copy = gst_df_copy[gst_df_copy['CME_ActivityID'].str.contains("CME")]
gst_df_copy.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [36]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df_copy, cme_df_copy, left_on='CME_ActivityID', right_on='GST_ActivityID')
merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T01:54:00-IPS-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T02:47:00-MPC-001
2,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-19T03:10:00-IPS-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:10:00-IPS-001
3,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-20T02:50:00-IPS-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T02:50:00-IPS-001
4,2014-02-27T18:00:00-GST-001,2014-02-27 18:00:00+00:00,2014-02-27T16:10:00-IPS-001,2014-02-25T01:25:00-CME-001,2014-02-25 01:25:00+00:00,2014-02-27T16:10:00-IPS-001


In [40]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.shape
gst_df_copy.shape
cme_df_copy.shape

(1714, 3)

### Computing the time it takes for a CME to cause a GST

In [37]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df["timeDiff"] = merged_df["startTime_GST"] - merged_df["startTime_CME"]
# Verify that the new column was created correctly
merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T01:54:00-IPS-001,2 days 04:20:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T02:47:00-MPC-001,2 days 04:20:00
2,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-19T03:10:00-IPS-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:10:00-IPS-001,2 days 12:45:00
3,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-20T02:50:00-IPS-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T02:50:00-IPS-001,2 days 01:35:00
4,2014-02-27T18:00:00-GST-001,2014-02-27 18:00:00+00:00,2014-02-27T16:10:00-IPS-001,2014-02-25T01:25:00-CME-001,2014-02-25 01:25:00+00:00,2014-02-27T16:10:00-IPS-001,2 days 16:35:00


In [39]:
# Use describe() to compute the mean and median time 
merged_df["timeDiff"].describe()
# that it takes for a CME to cause a GST. 


count                           81
mean     2 days 17:14:14.814814814
std      0 days 20:28:43.710892614
min                1 days 07:22:00
25%                2 days 03:12:00
50%                2 days 12:45:00
75%                3 days 07:48:00
max                5 days 02:36:00
Name: timeDiff, dtype: object

### Exporting data in csv format

In [42]:
# Export data to CSV without the index
merged_df.to_csv("./output/cme_gst.csv", index=False)